In [1]:
import  os

In [2]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:str
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import  create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import  os
import urllib.request as request
import zipfile
import gdown
from cnnClassifier import  logger
from cnnClassifier.utils.common import read_yaml

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try:
            data_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir,exist_ok=True)
            logger.info(f"Downloading data from {data_url} into file {zip_download_dir}")

            file_id = data_url.split('/')[-2]
            prefix = f'https://drive.google.com/uc?/export=download&id={file_id}'
            gdown.download(prefix,zip_download_dir)

            logger.info(f"Download data from {data_url} into file {zip_download_dir}")
        except Exception as e:
            logger.info(f"Dowload file from {self.config.source_URL} failed")
            raise e


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-02-15 12:53:27,974]: INFO: common: Yaml file: config\config.yml loaded successfully
[2026-02-15 12:53:28,013]: INFO: common: Yaml file: params.yml loaded successfully
[2026-02-15 12:53:28,018]: INFO: common: Created directory at: artifacts
[2026-02-15 12:53:28,021]: INFO: common: Created directory at: artifacts/data_ingestion
[2026-02-15 12:53:28,028]: INFO: 2216180264: Downloading data from https://drive.google.com/file/d/1Fo-0PhEG6DtQY2qFFW4GWvt2lg7nis9O/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Fo-0PhEG6DtQY2qFFW4GWvt2lg7nis9O
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Fo-0PhEG6DtQY2qFFW4GWvt2lg7nis9O&confirm=t&uuid=13ec9bd2-ac41-4f06-8faf-78610f9148f6
To: C:\Users\peguy\PycharmProjects\kidneydiseaseklassification\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [01:27<00:00, 10.7MB/s] 

[2026-02-15 12:54:59,149]: INFO: 2216180264: Download data from https://drive.google.com/file/d/1Fo-0PhEG6DtQY2qFFW4GWvt2lg7nis9O/view?usp=sharing into file artifacts/data_ingestion/data.zip
